In [2]:
# from google.colab import drive

In [3]:
# drive.mount('/content/gdrive')

In [4]:
# import os

In [5]:
# os.chdir("./gdrive/")
# os.chdir("./MyDrive/")
# os.chdir("./Hack_uns")

In [11]:
import torch
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
from tqdm import tqdm
import torchsummary as summary
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
import torch.nn.functional as F
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import time
import copy
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [15]:
data_dir = './datasets'
classes = os.listdir(data_dir)
classes

['acanthosis-nigricans',
 'acne',
 'acne-scars',
 'alopecia-areata',
 'dry',
 'melasma',
 'oily',
 'vitiligo',
 'warts']

In [17]:
def get_transforms(mode='train'):
    transform = []
    transform.append(T.Resize((512, 512)))
    transform.append(T.ToTensor())
#     transform.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transform)

In [19]:
dataset = ImageFolder(data_dir, transform=get_transforms())
dataset

Dataset ImageFolder
    Number of datapoints: 32
    Root location: ./datasets
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [20]:
len(dataset)

32

In [21]:
img, label = dataset[2]
print(img.shape, label)
img

torch.Size([3, 512, 512]) 1


tensor([[[0.7333, 0.7765, 0.8275,  ..., 0.7451, 0.7412, 0.7412],
         [0.7529, 0.7922, 0.8353,  ..., 0.7412, 0.7373, 0.7333],
         [0.7608, 0.7922, 0.8235,  ..., 0.7373, 0.7294, 0.7255],
         ...,
         [0.0902, 0.0902, 0.0902,  ..., 0.6078, 0.6039, 0.6039],
         [0.0902, 0.0902, 0.0902,  ..., 0.6078, 0.6039, 0.6000],
         [0.0902, 0.0902, 0.0902,  ..., 0.6078, 0.6000, 0.6000]],

        [[0.4353, 0.4824, 0.5412,  ..., 0.4196, 0.4157, 0.4157],
         [0.4510, 0.4941, 0.5451,  ..., 0.4157, 0.4118, 0.4078],
         [0.4431, 0.4824, 0.5255,  ..., 0.4118, 0.4039, 0.4000],
         ...,
         [0.0510, 0.0510, 0.0510,  ..., 0.3216, 0.3176, 0.3176],
         [0.0510, 0.0510, 0.0510,  ..., 0.3216, 0.3176, 0.3137],
         [0.0510, 0.0510, 0.0510,  ..., 0.3216, 0.3137, 0.3137]],

        [[0.3529, 0.4078, 0.4706,  ..., 0.2941, 0.2902, 0.2902],
         [0.3725, 0.4157, 0.4745,  ..., 0.2902, 0.2863, 0.2824],
         [0.3608, 0.4000, 0.4431,  ..., 0.2863, 0.2784, 0.

In [22]:
random_seed = 42
torch.manual_seed(random_seed);
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(26, 6)

In [23]:
efficientnet_b0 = models.efficientnet_b0(pretrained=True)

c:\Users\user\Desktop\Lucere\ML\workingml\Skin-Disease-Detection\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\Desktop\Lucere\ML\workingml\Skin-Disease-Detection\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\user/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100.0%


In [24]:
efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [25]:
efficientnet_b0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=9, bias=True)
)

In [26]:
efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [27]:
model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

np.int64(4019077)

In [28]:
out = efficientnet_b0(torch.randn((1,3,512,512)))
len(out[0])

9

In [29]:
out

tensor([[ 0.0207, -0.0846,  0.0280,  0.0180, -0.0956,  0.0189,  0.0072, -0.0170,
          0.0636]], grad_fn=<AddmmBackward0>)

In [30]:
for param in efficientnet_b0.features[:7].parameters():
    param.requires_grad = False

In [31]:
model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

np.int64(1140921)

In [32]:
params

np.int64(1140921)

In [33]:
from torch.utils.data.dataloader import DataLoader

batch_size=32

In [34]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

In [35]:
dataloaders = {'train':train_dl, 'val':val_dl}

In [36]:
dataset_sizes = {'train': len(train_ds), 'val': len(val_ds)}
dataset_sizes

{'train': 26, 'val': 6}

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [39]:
model = efficientnet_b0.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [40]:
# Fn for the training loop
def train_model(model, optimizer, alpha=1, beta=1, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print("-"*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            if phase == 'val':
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for imgs, labels in dataloaders[phase]:
                imgs = imgs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imgs)
                    _, preds = torch.max(outputs, 1)
                    loss = F.cross_entropy(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * imgs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            writer.add_scalar("Loss/train", epoch_loss, epoch)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [41]:
model = train_model(model, optimizer, num_epochs=25)

Epoch 1/25
----------
train Loss: 2.2867 Acc: 0.0385
val Loss: 2.1794 Acc: 0.1667

Epoch 2/25
----------
train Loss: 2.2639 Acc: 0.0769
val Loss: 2.1757 Acc: 0.1667

Epoch 3/25
----------
train Loss: 2.1829 Acc: 0.0769
val Loss: 2.1786 Acc: 0.1667

Epoch 4/25
----------
train Loss: 2.0827 Acc: 0.4615
val Loss: 2.1858 Acc: 0.1667

Epoch 5/25
----------
train Loss: 1.9938 Acc: 0.4231
val Loss: 2.1927 Acc: 0.1667

Epoch 6/25
----------
train Loss: 1.8194 Acc: 0.4615
val Loss: 2.1958 Acc: 0.1667

Epoch 7/25
----------
train Loss: 1.6934 Acc: 0.5769
val Loss: 2.1988 Acc: 0.1667

Epoch 8/25
----------
train Loss: 1.5702 Acc: 0.5000
val Loss: 2.2019 Acc: 0.1667

Epoch 9/25
----------
train Loss: 1.4304 Acc: 0.6154
val Loss: 2.2075 Acc: 0.0000

Epoch 10/25
----------
train Loss: 1.3619 Acc: 0.5769
val Loss: 2.2159 Acc: 0.0000

Epoch 11/25
----------
train Loss: 1.2551 Acc: 0.7308
val Loss: 2.2258 Acc: 0.0000

Epoch 12/25
----------
train Loss: 1.1421 Acc: 0.7308
val Loss: 2.2363 Acc: 0.0000

E

In [42]:
torch.save(model, "./skin-model-pokemon.pt")